# Data Preparation

To start with, we'll present some data preparation. We've downloaded the GraphQL from Meetups.com for this event and we can start loading this and transforming the data for our purposes.

In [ ]:
%pip install -q duckdb pandas pyarrow matplotlib

In [ ]:
from pathlib import Path
import logging
import codecs
import json
from datetime import datetime

# import logging properly.
logging.basicConfig(format='%(asctime)s:%(name)-35s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S %z')
logging.getLogger("OAuthUtil").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

is_debug = logging.getLogger().getEffectiveLevel() == logging.DEBUG

## Parse GraphQL from Meetups.com

We've downloaded the GraphQL from Meetups.com for AICamp which has a trove of information.

We'll now parse it into a structure we can use to load our corpus.

In [ ]:
meetups = []

for input in Path("../resources/meetups/aicamp").glob("*.json"):
    logger.info(f"Found file [{input.name}]")

    with codecs.open(input, encoding="utf-8") as f:
        content = json.loads(f.read())

    edges = content['data']['groupByUrlname']['events']['edges']
    for edge in edges:
        event = edge['node']
        title = event['title']
        event_datetime = datetime.strptime(event['dateTime'], "%Y-%m-%dT%H:%M:%S%z")
        event_date_str = datetime.strftime(event_datetime, '%Y/%m/%d')
        num_going = event['going']['totalCount']

        description_lines = event['description'].split("\n")

        skip_contents = [
            "Pre-registration is required, please complete RSVP here:",
            "https://www.aicamp.ai/event/eventdetails",
            "Welcome to our monthly in-person AI meetup in Sydney."
        ]
        
        content_lines = []

        for idx, line in enumerate(description_lines):
            skip = False
            for skip_content in skip_contents:
                if skip_content in line:
                    skip = True
                    break
            if skip:
                continue

            content_lines.append(line)
        description = "\n".join(content_lines)

        if is_debug:
            logger.debug(f"Description is:\n{description}")
        
                
        meetup = {
            "id": event['id'],
            "title": title,
            "description": description,
            "event_date": event_date_str,
            "event_year": datetime.strftime(event_datetime, '%Y'),
            "event_month": datetime.strftime(event_datetime, '%m'),
            "url": event['eventUrl'],
            "event_type": event['eventType'].lower(),
            "is_online": event['isOnline'],
            "num_going": num_going 
        }
        meetups.append(meetup)

    
logger.info(f"Total meetups is {len(meetups)}")
    

## View the Data

We'll use DuckDB to review the data we've downloaded.

In [ ]:
import pandas as pd
import duckdb
import pyarrow as pa

columns = []
for key in meetups[0].keys():
    columns.append(key)
    
df = pd.DataFrame(meetups, columns=columns)
con = duckdb.connect()
meetups_raw = pa.Table.from_pandas(df)

## Lets Look at Attendance

Lets look at how attendance has gone over the years!!

In [ ]:
result_df = con.execute("""SELECT event_year || '-' || event_month as event_year_month, cast(round(avg(num_going),0) as int) AS AVG_ATTENDANCE
FROM meetups_raw
GROUP BY event_year, event_month
ORDER BY event_year, event_month""").fetchdf()

#%matplotlib inline
result_df.plot(x="event_year_month")

## Lets look for Seasonality

We'll quickly review attendance by month to see how we do in May!!

In [ ]:
month_df = con.execute("""
SELECT 
    event_month, 
    cast(round(avg(num_going),0) as int) AS AVG_ATTENDANCE
FROM meetups_raw
GROUP BY event_month
ORDER BY event_month""").fetchdf()

month_df.plot(x="event_month")


## Persist the Output
We'll now persist the output to retain our early curation work for the next notebook.

In [ ]:
output_dir = "../output"
Path(output_dir).mkdir(parents=True, exist_ok=True)
con.execute(f"""COPY
    (SELECT * FROM meetups_raw)
    TO '{output_dir}/meetups_raw.parquet'
    (FORMAT 'parquet');""")